In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

# テキスト解析
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

In [2]:
df = pd.read_csv('all_text_wakatigaki.csv',encoding='utf-8')
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


### テキスト毎にわかち書きされた単語を結合
参考URL：https://github.com/Salinger/found_it_project_06_crwl/blob/master/src/python_crawler_nlp.ipynb

In [3]:
# df.groupbyでグループ化された[内容]毎の[original]カラムの各要素に対して文字列のリスト（配列）を一つの文字列に連結する「join」関数を各行（各要素）に対して実行する
# join関数の使い方: '間に挿入する文字列'.join([連結したい文字列のリスト])
text_wakati_df = df.groupby('内容')['original'].apply(lambda x: ' '.join(map(str, x))).reset_index()

# カラム名を適切な名前に変更
text_wakati_df = text_wakati_df.rename(columns={'original':'wakati_words'})

# 確認
text_wakati_df

,内容,wakati_words
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd _ tagger . Parse ( text ) で 各 単語 の 原形 ...
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,まずは Parse () で 分かち書き する た 単語 群 は 1つ の 文字列 型 に ...
2,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,原形 、 品詞 など の 間 に は 「 \ t 」 が 、 分かち書き する れる た 単...
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,次に 邪魔 だ 文字 「\ t 」 を 省く たい 。 よって 、 split ('\ t ...


### TF-IDFの実装(TfidfVectorizerを使用)
参考URL：https://github.com/hwpwk/NLP/blob/master/TfidfVectorizer%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E3%81%AE%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E5%8C%96(TF-IDF)%2BDataFrame%E5%8C%96_parse%E3%81%A7%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90_181030.ipynb

In [4]:
# 初期化
# 変更(181121)
# CountVectorizer や TfidfVectorizer は、デフォルトでは、一文字のトークンが除外されてしまうのでtoken_pattern=u'(?u)\\b\\w+\\b'を除外を阻止
# TfidfVectorizerではデフォルトで英字の大文字は小文字に変換するのでlowercase=Falseを引数に与え大文字を大文字として処理
# 参考URL：https://qiita.com/m__k/items/709a9cae184769e2243f
vectorizer= TfidfVectorizer(token_pattern=u'(?u)\\b\\w+\\b', lowercase=False)


# TfidfVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
tfidf_feature_vectors = vectorizer.fit_transform(text_wakati_df['wakati_words'])

# どの単語を学習しているのかをvectorizer.get_feature_names()で確認できる
# 後にデータフレームを作成する際にカラムとして使用する準備
tfidf_vocabulary = vectorizer.get_feature_names()

## 学習単語とIDFを紐付け

In [5]:
# TfidfVectorizer関数後にfit_transformしたあとに_tfidf.idf_にアクセスすればIDF値を取得できる
# 参考URL：https://qiita.com/m__k/items/22474ef67f07bc73602a
idf = vectorizer._tfidf.idf_
idf

array([1.91629073, 1.91629073, 1.91629073, 1.51082562, 1.51082562,
       1.51082562, 1.91629073, 1.51082562, 1.91629073, 1.51082562,
       1.91629073, 1.91629073, 1.22314355, 1.        , 1.51082562,
       1.91629073, 1.91629073, 1.91629073, 1.51082562, 1.22314355,
       1.91629073, 1.91629073, 1.91629073, 1.51082562, 1.91629073,
       1.51082562, 1.        , 1.91629073, 1.22314355, 1.91629073,
       1.91629073, 1.51082562, 1.91629073, 1.51082562, 1.91629073,
       1.51082562, 1.91629073, 1.91629073, 1.22314355, 1.51082562,
       1.91629073, 1.51082562, 1.51082562, 1.91629073, 1.51082562,
       1.91629073, 1.91629073, 1.91629073, 1.91629073, 1.51082562,
       1.91629073, 1.91629073, 1.91629073, 1.91629073])

In [6]:
# 紐付け方法1
# 単純にzip関数で学習単語とそのIDFをfor文で表示
for word, idf in zip(vectorizer.get_feature_names(), vectorizer._tfidf.idf_):
    print(str(word) + ':' + str(idf))

1:1.916290731874155
1つ:1.916290731874155
NEologd:1.916290731874155
Parse:1.5108256237659907
_:1.5108256237659907
n:1.5108256237659907
split:1.916290731874155
t:1.5108256237659907
tagger:1.916290731874155
text:1.5108256237659907
wakatid:1.916290731874155
いる:1.916290731874155
が:1.2231435513142097
する:1.0
た:1.5108256237659907
たい:1.916290731874155
ため:1.916290731874155
だ:1.916290731874155
て:1.5108256237659907
で:1.2231435513142097
できる:1.916290731874155
と:1.916290731874155
ない:1.916290731874155
など:1.5108256237659907
なる:1.916290731874155
に:1.5108256237659907
の:1.0
ので:1.916290731874155
は:1.2231435513142097
まずは:1.916290731874155
よって:1.916290731874155
れる:1.5108256237659907
を:1.916290731874155
リスト:1.5108256237659907
使用:1.916290731874155
分かち書き:1.5108256237659907
区切り:1.916290731874155
区切る:1.916290731874155
単語:1.2231435513142097
原形:1.5108256237659907
各:1.916290731874155
品詞:1.5108256237659907
型:1.5108256237659907
文字:1.916290731874155
文字列:1.5108256237659907
次に:1.916290731874155
省く:1.916290731874155
群:1.9

In [7]:
# 紐付け方法2
# 学習単語とそのIDFを辞書型に格納
# 参考URL：https://qiita.com/m__k/items/22474ef67f07bc73602a
word_idf_dict = {}
for word, idf in zip(vectorizer.get_feature_names(), vectorizer._tfidf.idf_):
    word_idf_dict[word] = idf

word_idf_dict

{'1': 1.916290731874155,
 '1つ': 1.916290731874155,
 'NEologd': 1.916290731874155,
 'Parse': 1.5108256237659907,
 '_': 1.5108256237659907,
 'n': 1.5108256237659907,
 'split': 1.916290731874155,
 't': 1.5108256237659907,
 'tagger': 1.916290731874155,
 'text': 1.5108256237659907,
 'wakatid': 1.916290731874155,
 'いる': 1.916290731874155,
 'が': 1.2231435513142097,
 'する': 1.0,
 'た': 1.5108256237659907,
 'たい': 1.916290731874155,
 'ため': 1.916290731874155,
 'だ': 1.916290731874155,
 'て': 1.5108256237659907,
 'で': 1.2231435513142097,
 'できる': 1.916290731874155,
 'と': 1.916290731874155,
 'ない': 1.916290731874155,
 'など': 1.5108256237659907,
 'なる': 1.916290731874155,
 'に': 1.5108256237659907,
 'の': 1.0,
 'ので': 1.916290731874155,
 'は': 1.2231435513142097,
 'まずは': 1.916290731874155,
 'よって': 1.916290731874155,
 'れる': 1.5108256237659907,
 'を': 1.916290731874155,
 'リスト': 1.5108256237659907,
 '使用': 1.916290731874155,
 '分かち書き': 1.5108256237659907,
 '区切り': 1.916290731874155,
 '区切る': 1.916290731874155,
 '単語': 1